## 4.1 텐서

### 4.1.1 Scalar

In [1]:
import torch

sca1 = torch.tensor([1.])
print(sca1)

tensor([1.])


In [2]:
sca2 = torch.tensor([3.,3.])
print(sca2)

tensor([3., 3.])


In [6]:
print(sca1 + sca2)
print(torch.add(sca1, sca2))

tensor([4., 4.])
tensor([4., 4.])


In [7]:
print(sca1 - sca2)
print(torch.sub(sca1, sca2))

tensor([-2., -2.])
tensor([-2., -2.])


In [8]:
print(sca1 * sca2)
print(torch.mul(sca1, sca2))

tensor([3., 3.])
tensor([3., 3.])


In [9]:
print(sca1/sca2)
print(torch.div(sca1,sca2))

tensor([0.3333, 0.3333])
tensor([0.3333, 0.3333])


### 4.1.2 Vector

벡터는 하나의 값을 표현할 때 2개 이상의 수치로 표현한 것이다.
스칼라의 형태와 동일한 속성을 가지고 있지만 여러 수치 값을 이용하여 표현하는 방식이다.

In [10]:
vec1 = torch.tensor([1.,2.,3.])
print(vec1)

tensor([1., 2., 3.])


In [11]:
vec2 = torch.tensor([4., 5., 6.])
print(vec2)

tensor([4., 5., 6.])


In [12]:
print(torch.dot(vec1, vec2))

tensor(32.)


벡터도 텐서와 마찬가지로 사칙연산이 가능하다.

### 4.1.3 행렬

행렬은 두개 이상의 벡터값을 통합해 구성된 값으로 벡터 값 간 연산 속도를 빠르게 진행할 수 있는 선형 대수의 기본 단위이다.

In [13]:
mat1 = torch.tensor([[1,2],[3,4]])
print(mat1)

tensor([[1, 2],
        [3, 4]])


In [15]:
mat2 = torch.tensor([[5,6],[7,8]])
print(mat2)

tensor([[5, 6],
        [7, 8]])


In [26]:
torch.matmul(mat1, mat2) # 행렬 곱 연산

tensor([[19, 22],
        [43, 50]])

행렬도 마찬가지로 사칙 연산이 가능하다.

### 4.1.4 텐서

행렬을 2차원의 배열이라고 표시할 수 있다면 텐서는 2차원 이상의 베열을 표현할 수 있다.

In [22]:
ten1 = torch.tensor([[[1.,2.],[3.,4.],[5.,6.],[7.,8.]]])
print(ten1)

tensor([[[1., 2.],
         [3., 4.],
         [5., 6.],
         [7., 8.]]])


In [23]:
ten2 = torch.tensor([[[9.,10.],[11.,12.],[13.,14.],[15.,16.]]])
print(ten2)

tensor([[[ 9., 10.],
         [11., 12.],
         [13., 14.],
         [15., 16.]]])


In [27]:
# 왜 안돼 ?
torch.matmul(ten1, ten2)

RuntimeError: Expected batch2_sizes[0] == bs && batch2_sizes[1] == contraction_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

## 4.2 Autograd

파이토치를 이용해 코드를 작성할 때 Back Propagation을 이용해 파라미터를 업데이트 하는 방법은 Autograd 방식으로 쉽게 구현할 수 있도록 설정되어 있다. 요기에서는 정말 간단한 딥러닝 모델을 설계하여 방정식 내에 존재하는 파라미터가 어떻게 업데이트할 수 있는지 알아보자.

autograd 패키지는 텐서의 모든 연산에 대해 자동미분을 제공한다. 이 패키지는 실행 시점에서 정의되는 프레임워크인데 코드가 어떻게 실행되는지에 따라 역전파가 정의되며 각각의 반복마다 역전파가 달라질 수 있다는 것이다.

In [29]:
if torch.cuda.is_available() :
    DEVICE = torch.device('cuda')
else :
    DEVICE = torch.device('cpu')
    
    # 참이면 cuda 장비를 이용한다.

In [31]:
batch_size = 64 # 파라미터를 업데이트 할 때 계산되는 데이터의 개수
input_size = 1000 # 딥러닝 모델에서의 입력층의 노드
hidden_size = 100 # input을 다수의 파라미터를 이용해 계산한 결과에 한 번 더 계산되는 파라미터 수
output_size = 10 # 딥러닝 모델에서 최종으로 출력되는 값의 벡터의 크기

batch_size 수만큼 데이터를 이용해 output을 계산하고 batch_size 수만큼 출려된 결괏값에 대한 오차값을 계산한다. batch_size 수만큼 계산된 오차값을 평균해 back propagation을 적용하고 이를 바탕으로 파라미터를 업데이트 한다. <br>
batch_size : 우리가 사용한 예제는 input으로 이용되는 데이터가 64개라는 것을 의미한다. <br>
input_size : 입력 데이터의 크기가 1000이라는 것이다. 1000 크기의 벡터의 값 <br>
=> (64,1000)<br>
hidden_size : 입력층에서 은닉층로 전달됐을 때 은닉층의 노드 수<br>
output_size : output의 크기는  최종으로 비교하고자 하는 레이블의 크기와 동일하게 설정

In [43]:
# requires_grad : 이미 존재하는 텐서의 requires_grad 플래그를 제자리에서 변경한다.
x = torch.randn(batch_size, input_size, device=DEVICE, dtype=torch.float, requires_grad=False)
y = torch.randn(batch_size, output_size, device=DEVICE, dtype=torch.float, requires_grad=False)
w1 = torch.randn(input_size, hidden_size, device=DEVICE, dtype=torch.float, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=DEVICE, dtype=torch.float, requires_grad=True)

(1) x 에 대한 설명

randn은 평균이 0 표준편차가 1인 정규분포에서 샘플링한 값으로 데이터를 만든다는 것이다. 이때 크기가 1000짜리 벡터를 64개 만들기 위해 사이즈를 설정하였으며 x 는 (64, 1000)의 데이터가 생성된다. 또한 우리는 파라미터 값을 업데이트하기 위해 Gradient를 계산하는 것이지 input에 대해 Gradient를 하는 것이 아니기 때문에 requires_grad를 False로 설정한다. 

(2) y 에 대한 설명

output 역시 batch_size 수만큼 결괏값이 필요하며 output과의 오차를 계산하기 위해 10으로 설정하였다.

(3) w1 에 대한 설명

업데이트할 파라미터 값을 설정한다. input의 데이터  크기가 1000이며 행렬 곱을 하기 위해 다음의 행 값이 1000이어야 한다. 또한 행렬 곱을 통해 100 크기의 데이터를 생성하기 위해 (1000, 100) 크기의 데이터를 생성한다. 또한 Gradient를 계산할 수 있도록 requires_grad를 True로 설정한다.

(4) w2 에 대한 설명

w1과 x를 행렬 곱한 결과에 계산할 수 있는 데이터여야 한다. 

In [44]:
learning_rate = 1e-6
for t in range(1,501):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 0 :
        print("Iteration:", t, "|t","Loss :", loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

Iteration: 100 |t Loss : 449.9139404296875
Iteration: 200 |t Loss : 1.283437728881836
Iteration: 300 |t Loss : 0.005996609106659889
Iteration: 400 |t Loss : 0.0001590217580087483
Iteration: 500 |t Loss : 3.321874828543514e-05


learning_rate : 딥러닝 모델에서 파라미터 값을 업데이트 할 때 가장 중요한 하이퍼파라미터이다.<br>
x.mm(x1) : 행렬 곱을 이용해 나온 결과값을 계산한다.<br>
clamp(min=0) : 비선형 함수, ReLU()와 같은 역할인데 최솟값이 0이며 0보다 큰 값은 자기 자신을 갖게 되는 메서드이다.clamp를 이용해 계산된 결과와 w2의 행렬 곱을 통해 나온 결과값을 도출한다.<br>
loss : 예측값 - 실제값의 제곱의 합<br>
loss.backward() : 각 파라미터 값에 대해 Gradient를 계산하고 이를 통해 Back Propagation을 진행한다는 의미이다.<br>
torch.no_grad() : 각 파라미터 값에 대해 Gradient를 계산한 결과를 이용해 파라미터 값을 업데이트할 때는 해당 시점의 Gradient 값을 고정한 후에 업데이트를 진행한다.


w1의 Gradient 값을 의미하는 w1.grad에 learning_rate를 곱한 값을 기존 w1에서 뻬준다. 음수를 이용하는 이유는 loss 값이 최소로 계산될 수 있는 파라미터 값을 찾기 위해 반대 방향으로 계산한다는 것을 의미한다.<br>
.zero_() : 다음 반복문을 진행할 수 있도록 Gradient 값을 0으로 설정한다. 다음 Backpropagation을 진행할 때 gradient 값을 loss,backward()를 통해 계산하기 때문이다.